In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as dset
from torchvision import datasets, transforms

In [ ]:
BATCH_SIZE = 256

In [ ]:
# GPU
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print('GPU State:', device)

GPU State: cuda:0


In [ ]:
# Transform
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,)),]
)

In [ ]:


# Data
trainSet = datasets.CIFAR10(root='CIFAR10', download=True, train=True, transform=transform)
testSet = datasets.CIFAR10(root='CIFAR10', download=True, train=False, transform=transform)
trainLoader = dset.DataLoader(trainSet, batch_size=BATCH_SIZE, shuffle=True)
testLoader = dset.DataLoader(testSet, batch_size=BATCH_SIZE, shuffle=False)

100%|██████████| 170498071/170498071 [00:05<00:00, 28534399.90it/s]


Extracting CIFAR10/cifar-10-python.tar.gz to CIFAR10
Files already downloaded and verified


In [ ]:
for batch_idx, (data, target) in enumerate(trainLoader):
    batch_size = data.size()
    print("Batch size:", batch_size[0])
    print("Color Channels:", batch_size[1])
    print("Image size:", batch_size[2],'x', batch_size[3])
    break

Batch size: 256
Color Channels: 3
Image size: 32 x 32


In [ ]:
# Model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()#120, 3, 32, 32
        self.main = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=4),
            nn.MaxPool2d(kernel_size=(4,4)),
            nn.Flatten(),
            nn.Linear(in_features=1568, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=10),
            nn.Softmax(dim=1)
        )

    def forward(self, input):
        return self.main(input)


net = Net().to(device)
print(net)

Net(
  (main): Sequential(
    (0): Conv2d(3, 32, kernel_size=(4, 4), stride=(1, 1))
    (1): MaxPool2d(kernel_size=(4, 4), stride=(4, 4), padding=0, dilation=1, ceil_mode=False)
    (2): Flatten(start_dim=1, end_dim=-1)
    (3): Linear(in_features=1568, out_features=128, bias=True)
    (4): ReLU()
    (5): Linear(in_features=128, out_features=64, bias=True)
    (6): ReLU()
    (7): Linear(in_features=64, out_features=10, bias=True)
    (8): Softmax(dim=1)
  )
)


# Train

In [ ]:
# Train
from matplotlib import pyplot as plt
def train():
  loss4graph = []
  for epoch in range(epochs):
      running_loss = 0.0

      for times, data in enumerate(trainLoader):
          inputs, labels = data[0].to(device), data[1].to(device)

          # Zero the parameter gradients
          optimizer.zero_grad()

          # Foward + backward + optimize
          outputs = net(inputs)

          #break
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          # Print statistics
          running_loss += loss.item()
          if times % 10000 == 9999 or times+1 == len(trainLoader):
              print("Epoch:", epoch+1, "  Loss:", round(running_loss,2))
              #print('[%d/%d, %d/%d] loss: %.3f' % (epoch+1, epochs, times+1, len(trainLoader), running_loss/2000))
              loss4graph.append(running_loss)
      scheduler.step()

  print('Training Finished.')
  plt.plot(loss4graph)

In [ ]:
# Test
def test():

  correct = 0
  total = 0

  with torch.no_grad():
    #    for times, data in enumerate(trainLoader):

      for data in testLoader:
          inputs, labels = data[0].to(device), data[1].to(device)
          #inputs = inputs.view(inputs.shape[0], -1)

          outputs = net(inputs)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  accuracies.append(100*correct / total)
  print(100*correct / total)


In [ ]:
#static parameters
accuracies = []
epochs = 15
learning_rates = [0.005, 0.01, 0.015, 0.02]

# Adjusting Parameters
for learning_rate in learning_rates:
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.7)
  scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9, verbose = True)
  train()
  test()

In [ ]:
max_accuracy = max(accuracies)
#max_lr = learning_rates[max_accuracy.index()]

stop_layer_index = 4  # Index of the layer before nn.Softmax
for idx, layer in enumerate(net.main):
    if idx == stop_layer_index:
        break
    print(layer)
print('\nAccuracy: %d %%' % max_accuracy)
#print('Best LR:', max_lr)

Conv2d(3, 32, kernel_size=(4, 4), stride=(1, 1))
MaxPool2d(kernel_size=(4, 4), stride=(4, 4), padding=0, dilation=1, ceil_mode=False)
Flatten(start_dim=1, end_dim=-1)
Linear(in_features=1568, out_features=128, bias=True)

Accuracy: 53 %


No Conv layer: 93%